In [ ]:
import cv2
import numpy as np
import glob

# 定義棋盤格的大小（內角點數量，9x6 的棋盤為 (9, 6)）
chessboard_size = (9, 6)
# 每個方格的物理尺寸（例如邊長 1 單位，可以是毫米、厘米等）
square_size = 1.0

# 準備棋盤格的 3D 坐標點（例如 (0,0,0), (1,0,0), ..., (8,5,0)）
objp = np.zeros((chessboard_size[0] * chessboard_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2) * square_size

# 儲存所有的物理和影像座標
objpoints = []  # 3D 坐標點（真實世界中的）
imgpoints = []  # 2D 坐標點（影像中的）

# 讀取包含棋盤格的所有影像
images = glob.glob('./calibration_images/*.jpg')  # 修改為你的影像資料夾路徑

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 嘗試在影像中找到棋盤格角點
    ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)

    if ret:
        objpoints.append(objp)  # 加入這張影像的 3D 點
        imgpoints.append(corners)  # 加入這張影像的 2D 點

        # 在影像中繪製角點，供檢查用
        cv2.drawChessboardCorners(img, chessboard_size, corners, ret)
        cv2.imshow('Chessboard', img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

# 使用所有影像的 objpoints 和 imgpoints 執行相機校正
ret, cameraMatrix, distCoeffs, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# 儲存校正結果
print("Camera Matrix:\n", cameraMatrix)
print("Distortion Coefficients:\n", distCoeffs)

# 保存為文件（例如 calibration_matrix.yaml 或 calibration_data.npz）
np.savez("calibration_data.npz", cameraMatrix=cameraMatrix, distCoeffs=distCoeffs, rvecs=rvecs, tvecs=tvecs)

# 確認校準誤差
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], cameraMatrix, distCoeffs)
    error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2) / len(imgpoints2)
    mean_error += error
print("Total Calibration Error: ", mean_error / len(objpoints))